# Importing Libraries & Data Preparation

In [ ]:
import numpy as np
import pandas as pd
from fastai import *
from fastai.text import *
from fastai.callbacks.tracker import EarlyStoppingCallback
from fastai.callbacks.tracker import SaveModelCallback

from clinical_note_utils import *

from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, BertTokenizer


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT")

In [ ]:
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [ ]:
DATA_ROOT = Path("./data")  

train, valid = [pd.read_csv(DATA_ROOT / fname) for fname in ["train_sub.csv", "valid.csv"]]

In [ ]:
train.head()

In [ ]:
valid.head()

In [ ]:
fastai_bert_vocab = Vocab(list(tokenizer.vocab.keys()))

In [ ]:
fastai_tokenizer = Tokenizer(tok_func=FastAiBertTokenizer(tokenizer, max_seq_len=256), pre_rules=[], post_rules=[])

In [ ]:
clinical_data = TextDataBunch.from_df(".", train, valid, 
                  tokenizer=fastai_tokenizer,
                  vocab=fastai_bert_vocab,
                  include_bos=False,
                  include_eos=False,
                  text_cols="TEXT",
                  label_cols="OUTPUT_LABEL",
                  bs=16,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )


In [ ]:
clinical_data.save("bert-clinical-sub.pkl")

In [ ]:
clinical_data.show_batch()

# BERT Model

In [ ]:
clinical_bert = BertForSequenceClassification.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT",
              num_labels = 2)

In [ ]:
class CustomTransformerModel(nn.Module):
  
    def __init__(self, transformer_model: BertForSequenceClassification):
        super(CustomTransformerModel,self).__init__()
        self.transformer = transformer_model
        
    def forward(self, x):
        # Return only the logits from the transfomer
        logits = self.transformer(x)[0]   
        return logits

In [ ]:
model = CustomTransformerModel(clinical_bert)

In [ ]:
from fastai.callbacks import *

learner = Learner(clinical_data, model, metrics=[AUROC(), Precision(), Recall()])

In [ ]:
def bert_clas_split(self) -> List[nn.Module]:
    
    bert = model.transformer.bert
    embedder = bert.embeddings
    pooler = bert.pooler
    encoder = bert.encoder
    classifier = [model.transformer.dropout, model.transformer.classifier]
    n = len(encoder.layer)//3
    print(n)
    groups = [[embedder], list(encoder.layer[:n]), list(encoder.layer[n+1:2*n]), list(encoder.layer[(2*n)+1:]), [pooler], classifier]
    return groups

x = bert_clas_split(model)
learner.split([x[0], x[1], x[2], x[3], x[4], x[5]])

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(3, max_lr=slice(1e-06, 1e-05), moms=(0.8, 0.7), wd =(1e-7, 1e-06, 1e-5, 1e-4, 1e-3, 1e-02))

In [ ]:
learner.save("clinical-bert-1-b")

In [ ]:
learner.freeze_to(-2)

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(10, max_lr=slice(1e-05, 1e-04), moms=(0.8, 0.7), pct_start=0.2, wd =(1e-7, 1e-06, 1e-5, 1e-4, 1e-3, 1e-02))

In [ ]:
learner.save("clinical-bert-2")

In [ ]:
learner.freeze_to(-3)

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(3, max_lr=slice(1e-6, 1e-05), moms=(0.8, 0.7), pct_start=0.1, wd =(1e-7, 1e-06, 1e-5, 1e-4, 1e-3, 1e-02))

In [ ]:
learner.save("clinical-bert-3")

In [ ]:
learner.unfreeze()

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(2, max_lr=slice(1e-05, 1e-04), moms=(0.8, 0.7), pct_start=0.1, wd =(1e-7, 1e-06, 1e-5, 1e-4, 1e-3, 1e-02))

In [ ]:
learner.save("clinical-bert-unfrozen-1")

In [ ]:
test = pd.read_csv(DATA_ROOT / "test.csv")
learner.data.add_test(test.TEXT)

In [ ]:
learner.data.test_ds

In [ ]:
preds = learner.get_preds(DatasetType.Test)

In [ ]:
final_preds = [1 if p > 0.5 else 0 for p in preds[0][:, 1]]

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(test.OUTPUT_LABEL, final_preds)